In [3]:
import pandas as pd
import numpy as np

In [4]:
# creating lists to use for later looping
years = [1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014, 2015,2016,2017,2018,2019]
leagues = ["english_premier_league","italian_serie_a","spanish_primera_division"]

In [5]:
# Creating a function that will merge the various years for each league chosen
def data_org(league):
    appended_data = []
    for year in years:
        file_path=f"../raw/{year}/{league}.csv"
        df_league = pd.read_csv(file_path)
        appended_data.append(df_league)
    new_df = pd.concat(appended_data)
    new_df = new_df[["year","club_name","transfer_movement","fee_cleaned"]]
    new_df = new_df[new_df["transfer_movement"]=='in']
    return new_df

In [6]:
# Applying the function to build our datasets
premier_league_df = data_org(leagues[0])
italian_df = data_org(leagues[1])
spanish_df = data_org(leagues[2])

In [7]:
# Creating a function that formats the dataframes to suit requirements for analysis
def group_transfers(df):
    df_group = df.groupby(["year"]).sum()
    df_group = df_group.sort_values(by=["year"],ascending=True).rename(columns={"fee_cleaned": "Transfer_Spend","year":"Year"}).reset_index()
    df_group = df_group.rename(columns={"club_name":"Teams"})
    return df_group

In [8]:
# Creating a function that formats the dataframes to suit requirements for analysis
def group_and_clean(df):
    df_group = df.groupby(["year","club_name"]).sum()
    df_group = df_group.sort_values(by=["year","fee_cleaned"],ascending=False).rename(columns={"fee_cleaned": "Transfer Spend (Euros)","year":"Year"}).reset_index()
    df_group = df_group.rename(columns={"club_name":"Teams"})
    return df_group

In [11]:
italian_summary

,year,Transfer_Spend
0,1992,107.738
1,1993,69.775
2,1994,34.088
3,1995,85.210
4,1996,101.566
5,1997,193.487
6,1998,300.232
7,1999,442.465
8,2000,449.642
9,2001,742.775


In [10]:
# Applying the function to format our datasets
italian_summary = group_transfers(italian_df)
english_summary = group_transfers(premier_league_df)
spanish_summary = group_transfers(spanish_df)

In [1]:
transfer_summary = pd.concat([italian_summary,english_summary,spanish_summary])


NameError: name 'pd' is not defined

In [37]:
# Applying the function to format our datasets
italian_transfers = group_and_clean(italian_df)
english_transfers = group_and_clean(premier_league_df)
spanish_transfers = group_and_clean(spanish_df)

In [39]:
english_summary["League"]="England"
italian_summary["League"]="Italy"
spanish_summary["League"]="Spain"

In [ ]:
years = [1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014, 2015,2016,2017,2018,2019]
leagues = ["english_premier_league","italian_serie_a","spanish_primera_division"]

if year >2013:

    for year in years:
    
    year[i]-year[i-1]-year[i-2]==2

In [40]:
# A function that will be used when calculating the moving average 
# In this data set, there are teams that have not been in the league for each year we are working with due to promotion / relegation. 
# This function ensures that we are only calculating the Moving Average for teams that have been there for consecutive years.

def year_test(time):
    count = 0
    length = len(time) - 1
    outcome = "False"
    # Using the difference between years to determine if they are sequential.
    for year1, year2 in zip(time,time[1:]):
        total = year2 - year1
        count += total
    if length == count:
        outcome = "True"
    return outcome

In [41]:
# Function to calculate the moving average (MA)
def moving_average(df):
    # empty list to store our dataframes
    append_team = []
    
    # sorting values by Team and then year to allow average to calculate
    df = df.sort_values(by=["Teams","year"], ascending = True)
    
    # generating a unique list of the teams
    teams = df["Teams"].unique().tolist()

    for team in teams:
        # filtering on an individual team to create their 3yr MA
        team_df = df[df["Teams"]==team]
        years = team_df["year"].tolist()
        if year_test(years) == "True":
            # using 'rolling' function to create MA
            team_df["MA"] = team_df.iloc[:,2].rolling(window=3,min_periods = 3).mean().round(2)
            team_df = team_df[["year","Teams","MA"]]
            append_team.append(team_df)
    team_list = pd.concat(append_team)
    
    #joining the dataframes
    team_join = pd.merge(df,team_list,how = "outer",on=["year","Teams"])
    return team_join

In [42]:
# Applying MA function to each league
spanish_summary = moving_average(spanish_transfers)
english_summary = moving_average(english_transfers)
italian_summary = moving_average(italian_transfers)

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [43]:
# Cleaning Italian Data
italian_summary["Teams"] = italian_summary["Teams"].str.replace("\s$","")
italian_summary["Teams"] = italian_summary["Teams"].str.replace("\s+FC", "")
italian_summary["Teams"] = italian_summary["Teams"].str.replace("\s+BC", "")
italian_summary["Teams"] = italian_summary["Teams"].str.replace("\s+CFC", "")
italian_summary["Teams"] = italian_summary["Teams"].str.replace("^\s?UC", "")
italian_summary["Teams"] = italian_summary["Teams"].str.replace("^\s?US", "")
italian_summary["Teams"] = italian_summary["Teams"].str.replace("^\s?SSC\s?", "")
italian_summary["Teams"] = italian_summary["Teams"].str.replace("^\s?","")

In [44]:
# Cleaning English Data
english_summary["Teams"] = english_summary["Teams"].str.replace("\s$","")
english_summary["Teams"] = english_summary["Teams"].str.replace("\s+FC", "")
english_summary["Teams"] = english_summary["Teams"].str.replace("\s+AFC", "")
english_summary["Teams"] = english_summary["Teams"].str.replace("^AFC\s", "")

In [45]:
# Outputting data to CSV
italian_summary.to_csv("../cleaned/italian_transfers.csv")
english_summary.to_csv("../cleaned/english_transfers.csv")
spanish_summary.to_csv("../cleaned/spanish_transfers.csv")
transfer_summary.to_csv("../cleaned/transfers_summary.csv")